# Available variables in the ICON HAM-lite simulation

This notebook will demonstrate which variables are available in the HAM-lite model run stored on jasmin.

Note that this data is currently staged locally, so is not available on the catalogue and is still in the ICON grid format. A healpix version will be worked on this week!

In [1]:
import pathlib

import numpy as np
import pandas as pd
import xarray as xr

In [2]:
data_path = pathlib.Path("/work/scratch-pw2/wkjones/hamlite")

In [3]:
def get_hamlite_vars_df(data_path):
    atm_files = sorted(list(data_path.rglob("r2b9_lite*.nc")))
    ham_files = sorted(list(data_path.rglob("hamlite*.nc")))

    hamlite_files_df = pd.DataFrame(ham_files + atm_files, columns=["filename"])
    hamlite_files_df["file_type"] = ["_".join(f.name.split("_")[:-1]) for f in hamlite_files_df.filename]

    all_files = hamlite_files_df.groupby("file_type").first()
    
    vars_dict = {}

    for f in all_files.filename:
        print(f)
        file_type = "_".join(f.name.split("_")[:-1])
        with xr.open_dataset(f, chunks={}) as ds:
            vars_dict[file_type] = {
                var:dict(
                    standard_name=ds[var].standard_name if "standard_name" in ds[var].attrs else "", 
                    long_name=ds[var].long_name if "long_name" in ds[var].attrs else "", 
                    units=ds[var].units if "units" in ds[var].attrs else "",
                    size=ds[var].data.nbytes / 2**30, 
                    time_steps=ds[var].time.size if "time" in ds[var].coords else 1, 
                    levels=ds[var].height.size if "height" in ds[var].coords else (ds[var].height_2.size if "height_2" in ds[var].coords else 1), 
                    file_type=file_type, 
                ) for var in ds.data_vars if var not in ["height_bnds", "height_2_bnds", "clon_bnds", "clat_bnds"]
            }

    all_icon_hamlite_vars = pd.concat(
        pd.DataFrame(vd[1]).T for vd in vars_dict.items()
    )

    all_icon_hamlite_vars["frequency"] = ""
    all_icon_hamlite_vars.loc[all_icon_hamlite_vars.time_steps==1, "frequency"] = "P1D"
    all_icon_hamlite_vars.loc[all_icon_hamlite_vars.time_steps==24, "frequency"] = "PT1H"
    all_icon_hamlite_vars.loc[all_icon_hamlite_vars.time_steps==96, "frequency"] = "PT15M"

    all_icon_hamlite_vars["3d"] = all_icon_hamlite_vars.levels > 1

    return all_icon_hamlite_vars

In [4]:
vars_df = get_hamlite_vars_df(data_path)

/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_act_3d_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_act_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_bdn_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_ddep_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_emi_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_hab_du_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_hab_ta_pl_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_hab_wa_pl_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_20200120T000000-20200124T235920/hamlite_rad_3d_ml_20200120T000000Z.nc
/work/scratch-pw2/wkjones/hamlite/run_2

In [5]:
vars_df

,standard_name,long_name,units,size,time_steps,levels,file_type,frequency,3d
cdnc,cdnc,cdnc,m-3,3.984375,1,51,hamlite_act_3d_ml,P1D,True
base,base,level of cloud base,-,3.984375,1,51,hamlite_act_3d_ml,P1D,True
cdnc_mx,cdnc_mx,maximum cdnc in cloud levels,m-3,1.875,24,1,hamlite_act_ml,PT1H,False
cdnc_2d,cdnc_2d,average cdnc in cloud levels,m-3,1.875,24,1,hamlite_act_ml,PT1H,False
wa_mx,wa_mx,maximum updraft velocity,m s-1,1.875,24,1,hamlite_act_ml,PT1H,False
...,...,...,...,...,...,...,...,...,...
sse_hcap_grnd_old_box,heat_capacity_ground_old,Ground heat capacity (old),J m-2 K-1,0.078125,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_runoff_box,surface_runoff,Surface runoff,kg m-2 s-1,0.078125,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_drainage_box,drainage,drainage,kg m-2 s-1,0.078125,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_discharge_ocean_box,discharge_ocean,discharge to the ocean,m3 s-1,0.078125,1,1,r2b9_lite_0924g_lnd_ml,P1D,False


## Daily 3D data:

In [6]:
vars_df[vars_df["3d"]]

,standard_name,long_name,units,size,time_steps,levels,file_type,frequency,3d
cdnc,cdnc,cdnc,m-3,3.984375,1,51,hamlite_act_3d_ml,P1D,True
base,base,level of cloud base,-,3.984375,1,51,hamlite_act_3d_ml,P1D,True
abs_tot_550nm,abs_tot_550nm,absorption optical depth 550nm,1,3.984375,1,51,hamlite_rad_3d_ml,P1D,True
tau_tot_550nm,tau_tot_550nm,optical depth 550nm,1,3.984375,1,51,hamlite_rad_3d_ml,P1D,True
num_qdu,num_qdu,num_qdu,1 kg-1,3.984375,1,51,hamlite_trc_3d_ml,P1D,True
num_qss,num_qss,num_qss,1 kg-1,3.984375,1,51,hamlite_trc_3d_ml,P1D,True
num_qca,num_qca,num_qca,1 kg-1,3.984375,1,51,hamlite_trc_3d_ml,P1D,True
num_qsu,num_qsu,num_qsu,1 kg-1,3.984375,1,51,hamlite_trc_3d_ml,P1D,True
hus,specific_humidity,Specific humidity,kg kg-1,3.984375,1,51,r2b9_lite_0924g_atm_3d_cld_ml,P1D,True
clw,clw,specific cloud water content,kg kg-1,3.984375,1,51,r2b9_lite_0924g_atm_3d_cld_ml,P1D,True


## Daily 2D data:

In [7]:
vars_df[~vars_df["3d"] & (vars_df.frequency == "P1D")]

,standard_name,long_name,units,size,time_steps,levels,file_type,frequency,3d
ps,surface_air_pressure,surface pressure,Pa,0.078125,1,1,r2b9_lite_0924g_atm_3d_pre_ml,P1D,False
psl,mean sea level pressure,mean sea level pressure,Pa,0.078125,1,1,r2b9_lite_0924g_atm_3d_pre_ml,P1D,False
orog,surface_altitude,surface altitude,m,0.078125,1,1,r2b9_lite_0924g_atm_vgrid,P1D,False
ps,surface_air_pressure,surface pressure,Pa,0.078125,1,1,r2b9_lite_0924g_atm_vgrid,P1D,False
psl,mean sea level pressure,mean sea level pressure,Pa,0.078125,1,1,r2b9_lite_0924g_atm_vgrid,P1D,False
soil_depth_water_bnds,,,,0.0,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
soil_depth_energy_bnds,,,,0.0,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_w_soil_sl_box,Water content in soil layers,,m water equivalent,0.390625,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_w_ice_sl_box,Ice content in soil layers,,m water equivalent,0.390625,1,1,r2b9_lite_0924g_lnd_ml,P1D,False
hydro_w_soil_column_box,Water content in the whole soil column,,m water equivalent,0.078125,1,1,r2b9_lite_0924g_lnd_ml,P1D,False


## Hourly 2D data:

In [8]:
vars_df[~vars_df["3d"] & (vars_df.frequency == "PT1H")]

,standard_name,long_name,units,size,time_steps,levels,file_type,frequency,3d
cdnc_mx,cdnc_mx,maximum cdnc in cloud levels,m-3,1.875,24,1,hamlite_act_ml,PT1H,False
cdnc_2d,cdnc_2d,average cdnc in cloud levels,m-3,1.875,24,1,hamlite_act_ml,PT1H,False
wa_mx,wa_mx,maximum updraft velocity,m s-1,1.875,24,1,hamlite_act_ml,PT1H,False
wa_2d,wa_2d,average updraft velocity,m s-1,1.875,24,1,hamlite_act_ml,PT1H,False
base_2d,base_2d,average level of cloud base,-,1.875,24,1,hamlite_act_ml,PT1H,False
...,...,...,...,...,...,...,...,...,...
rlut_gmean,rlut_gmean,global mean toa outgoing longwave radiation,W m-2,0.0,24,1,r2b9_lite_0924g_atm_mon,PT1H,False
radtop_gmean,radtop_gmean,global mean toa total radiation,W m-2,0.0,24,1,r2b9_lite_0924g_atm_mon,PT1H,False
prec_gmean,prec_gmean,global mean precipitation flux,kg m-2 s-1,0.0,24,1,r2b9_lite_0924g_atm_mon,PT1H,False
evap_gmean,evap_gmean,global mean evaporation flux,kg m-2 s-1,0.0,24,1,r2b9_lite_0924g_atm_mon,PT1H,False


## 15 minute 2D data

In [9]:
vars_df[~vars_df["3d"] & (vars_df.frequency == "PT15M")]

,standard_name,long_name,units,size,time_steps,levels,file_type,frequency,3d
burden_num_qdu,burden_num_qdu,atmospheric burden of num_qdu,m-2,7.5,96,1,hamlite_bdn_ml,PT15M,False
burden_num_qss,burden_num_qss,atmospheric burden of num_qss,m-2,7.5,96,1,hamlite_bdn_ml,PT15M,False
burden_num_qca,burden_num_qca,atmospheric burden of num_qca,m-2,7.5,96,1,hamlite_bdn_ml,PT15M,False
burden_num_qsu,burden_num_qsu,atmospheric burden of num_qsu,m-2,7.5,96,1,hamlite_bdn_ml,PT15M,False
num_qdu,num_qdu,num_qdu,1 kg-1,7.5,96,1,hamlite_hab_du_ml,PT15M,False
ta,air_temperature,Temperature,K,7.5,96,1,hamlite_hab_ta_pl,PT15M,False
wa,upward_air_velocity,Vertical velocity,m s-1,7.5,96,1,hamlite_hab_wa_pl,PT15M,False
cllvi,total_cloud_water,vertically integrated cloud water,kg m-2,7.5,96,1,r2b9_lite_0924g_atm_2d_vid_ml,PT15M,False
clivi,total_cloud_ice,vertically integrated cloud ice,kg m-2,7.5,96,1,r2b9_lite_0924g_atm_2d_vid_ml,PT15M,False
pr,pr,precipitation flux,kg m-2 s-1,7.5,96,1,r2b9_lite_0924g_atm_2d_vid_ml,PT15M,False
